## Fees research
Based on DAI CDP protocol

There are two types of fees in DAI protocol
- Liquidation
- Borrow fee

We are interested in Borrow Fee(also called Variable Annual Fee).

A fundamental feature of the MCD system is to accumulate stability fees on Vault debt balances.

Example of how fee works:
> Stability Fee is set to 2%, it will accumulate at 0.000000000627937192491029810% per second. At the end of year one, the user will owe exactly 2% on the principal.

Assuming the user borrows 100 Dai and the rate doesn’t change, at the end of year one they would have a debt of 102.00 Dai. At the end of year two, they would have a debt of 104.04 Dai.

### Key points about borrow fees:
- Fees accrue over a year
- Different protocols(ETH-C, ETH-B, ETH-A, ...) have different stability fees based on collateral ratio, that is based on risk exposure. For example, for collateral 170.00% fee is 0.5%, while for collateral of 145.00% fee is larger - 1.5%

## Multiply feature(Leverage)

Oasis Multiply is a feature that allows you to immediately utilize your borrowed Dai to buy more collateral within Oasis.app. This means that you can increase or decrease your exposure to a single asset without having to perform multiple transactions or go to other apps to perform the exchange.

It sounds like a two-way borrow market


## Now, let's first model the case for plain borrowing with variable fees